In [ ]:
!git clone https://github.com/marco-c/crashcorrelations

In [ ]:
import os
import errno
import json
import gzip

from crashcorrelations import download_data
from crashcorrelations import crash_deviations

In [ ]:
channels = ['release', 'beta', 'aurora', 'nightly']

channel_to_versions = {}
results = {}

for channel in channels:
    results[channel] = {}
    channel_to_versions[channel] = download_data.get_versions(channel)

In [ ]:
# download_data.set_token('INSERT_YOUR_TOKEN_HERE')

for channel in channels:
    download_data.download_crashes(versions=channel_to_versions[channel], days=5)

In [ ]:
for channel in channels:
    crash_deviations.clear_cache('a')
    df_a = crash_deviations.get_crashes(sc, versions=channel_to_versions[channel], days=5)
    results[channel] = {
      'total': df_a.count(),
      'signatures': {},
    }
    signatures = download_data.get_top(200, channel_to_versions[channel], 11)
    i = 1
    for signature in signatures:
        print(channel + '-' + str(i) + ' ' + signature)
        i += 1
        res, total_a, total_b = crash_deviations.find_deviations(sc, df_a, signature=signature, cache_a=True)
        results[channel]['signatures'][signature] = {
          'total': total_b,
          'results': res,
        }

In [ ]:
try:
    os.mkdir('output')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise e

with gzip.open('output/top_results.json.gz', 'wb') as f:
    json.dump(results, f)